In [2]:
#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
#MY_HOME_ABS_PATH = "/Users/jetcalz07/Desktop/MIDS/W210_Capstone/co2-flux-hourly-gpp-modeling"
MY_HOME_ABS_PATH = "/root/co2-flux-hourly-gpp-modeling"

In [14]:
from tqdm import tqdm
from tempfile import TemporaryFile
import pandas as pd
import csv
import os
os.chdir(MY_HOME_ABS_PATH)
import sys
sys.path.append(os.path.abspath("./code/src/tools"))
from CloudIO.AzStorageClient import AzStorageClient
pd.set_option('display.max_columns', None)

## Define Constants

In [4]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if not (os.path.exists(tmp_dir)):
    print("Making tmp_dir")
    os.mkdir(tmp_dir)

In [5]:
# Initialize Azure Storage Client
azStorageClient = AzStorageClient(az_cred_file)
container_name = 'full-half-hourly-raw'
blob_name_list = azStorageClient.listBlobs(container_name)

## Pull Site Data

In [6]:
# Check one example to get colnames
ex_file = tmp_dir + os.sep + 'site_example_ZM-Mon.csv' 
site_ex = pd.read_csv(ex_file)
col_names = site_ex.columns
site_ex.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200003020000,200003020030,19.618,0,19.504,0.0,0.000,0,0.0,400.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
1,200003020030,200003020100,19.800,0,19.379,0.0,0.000,0,0.0,406.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
2,200003020100,200003020130,19.681,0,19.254,0.0,0.318,0,0.0,408.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
3,200003020130,200003020200,18.311,0,19.130,0.0,0.224,0,0.0,403.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate
4,200003020200,200003020230,17.640,0,19.005,0.0,0.789,0,0.0,399.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DBF,Temperate


In [7]:
# Download all HH site file to .tmp dir
for i, blob in tqdm(enumerate(blob_name_list)):
    blob_name = blob.name
    data = azStorageClient.downloadBlob(container_name, blob_name)
    
    # Create a temporary file to write the CSV file with column names
    with TemporaryFile(mode='w+', newline='') as tmp_file:
        writer = csv.writer(tmp_file)
        
        # check if the csv has the column names, if not add them in
        data_lines = data.decode().splitlines()
        reader = csv.reader(data_lines)
        header = next(reader)

        # Check if the first row contains the expected columns
        if tuple(header) == tuple(col_names):
            # If the first row matches col_names, write it as the header
            writer.writerow(col_names)
            data_start_idx = 1
        else:
            # If the first row does not match col_names, assume it is data and write it as is
            writer.writerow(col_names)
            data_start_idx = 0
        
        # Write the data as the remaining rows
        writer.writerows(csv.reader(data_lines[data_start_idx:]))
        
        # Save the temporary file to disk
        filename = os.path.join(tmp_dir, blob_name)
        with open(filename, "w", newline='') as f:
            tmp_file.seek(0)
            f.write(tmp_file.read())

276it [21:39,  4.71s/it]


In [15]:
# Check that the Azure df == the locally uploaded example from GDrive
local_filename = tmp_dir + os.sep + f'data_full_half_hourly_raw_v0_1_ZM-Mon.csv'  # <--- addd in at end to compare
local_filename = tmp_dir + os.sep + f'data_full_half_hourly_raw_v0_1_AT-Neu.csv' 
site_df = pd.read_csv(local_filename)
print(site_df.equals(site_ex))
display(site_df.head())

False


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,0,2.611260e+02,1.452,0,1.020,0.0000,0.000,0.0,91.300,0.00,91.186,-53.150,1.2271,-24.9018,0.000,3.0676,0.0,3.90238,-11.1307,1.0,-14.1597,8.68037,1.0000,5.47814,1.0,-5.134180,0.0,-4.271810,0.00000,3.21873,2.71260,1.20633,2.3933,2007-04-27 04:30:00,2007.0,4.000000,27.000000,4.00000,AT-Neu,2007-04-27,8.68037,NaN,0.405207,0.7356164455413818,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.18440,0.0906,GRA,Cold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200704270500,2.007043e+11,5.299,0,7.552,20.0592,7.224,0.0,4.647,265.12,0.000,261.126,1.5290,0.0000,1.050,0.0000,0.0,0.00000,91.3000,0.0,91.1900,-38.63150,14.6788,-27.02210,0.0,-0.498390,0.0,-0.634016,-14.34530,0.00000,-18.24910,8.68037,1.0000,5.47814,1.0,-5.074390,0.868053,-4.23522,0.704805,3.27674,2.77915,1.24292,2.439760,2007-04-27 05:00:00,2007.000000,4.0000,27.0000,5.0000,AT-Neu,2007-04-27,8.68037,NaN,0.4052065647990945,0.7356164455413818,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold
2,200704270530,2.007043e+11,5.732,0,7.613,126.0630,25.806,0.0,48.229,274.22,0.000,261.126,1.6250,0.0000,1.080,0.0000,0.0,0.00000,91.3000,0.0,91.1950,-23.96220,50.1543,-23.40660,0.0,-0.603316,1.0,-0.767496,-1.50852,1.00000,-1.91903,10.88960,2.0000,8.92647,2.0,-7.159060,2.990560,-7.62205,2.45353,3.39474,2.89366,1.30442,2.555570,2007-04-27 05:30:00,2007.000000,4.0000,27.0000,5.0000,AT-Neu,2007-04-27,NaN,NaN,0.4052065647990945,0.7356164455413818,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold
3,200704270600,2.007043e+11,5.955,0,7.674,243.2710,48.491,0.0,92.497,271.35,0.000,261.126,1.4700,0.0000,1.109,0.0000,0.0,0.09600,91.4000,0.0,91.1990,-16.87650,99.7732,-20.25680,0.0,-1.575570,0.0,-2.004330,2.77965,0.00000,3.53607,8.87825,2.0000,7.91275,2.0,-5.074140,5.383960,-6.56535,4.46741,3.46575,2.97168,1.34740,2.616880,2007-04-27 06:00:00,2007.000000,4.0000,27.0000,6.0000,AT-Neu,2007-04-27,NaN,NaN,0.4052065647990945,0.7356164455413818,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold
4,200704270630,2.007043e+11,6.861,0,7.734,360.3130,63.257,0.0,136.694,262.21,0.000,261.126,1.8030,0.0000,1.139,0.0000,0.0,0.09600,91.4000,0.0,91.2030,-18.99410,133.7970,-16.37970,0.0,0.235862,1.0,0.300047,8.78324,1.00000,11.17340,3.88074,2.0000,3.72161,2.0,0.174312,6.841330,-2.26164,5.71969,3.70134,3.18198,1.45997,2.873360,2007-04-27 06:30:00,2007.000000,4.0000,27.0000,6.0000,AT-Neu,2007-04-27,NaN,NaN,0.4052065647990945,0.7356164455413818,0.186405,0.0386,0.2534,0.0213,0.0482,0.2987,0.1844,0.0906,GRA,Cold


In [9]:
tmp_dir_size = len([x for x in os.listdir(tmp_dir) if 'data_full_half_hourly_raw' in x])
print(f"Number of site files in tmp_dir: {tmp_dir_size}")

Number of site files in tmp_dir: 276


In [22]:
local_filename = tmp_dir + os.sep + f'data_full_half_hourly_raw_v0_1_AT-Neu.csv' 
site_df = pd.read_csv(local_filename)
site_df = site_df.iloc[1:].copy()
site_df.head()

site_df.to_csv(local_filename)

/tmp/ipykernel_704/6168259.py:2: DtypeWarning: Columns (33,38,39,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  site_df = pd.read_csv(local_filename)
